In [32]:
import getpass
import requests
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display, HTML, JSON, Code, Markdown
import json
%reload_ext version_information
%version_information vega, pandas, IPython, ipywidgets, re

Software versions
Python 3.7.3 64bit [GCC 7.3.0]
IPython 7.8.0
OS Linux 5.0.0 32 generic x86_64 with debian buster sid
vega 0.5.0
pandas 0.25.1
IPython 7.8.0
ipywidgets 7.5.1
re 2.2.1
Wed Nov 20 11:26:06 2019 UTC

In [2]:
token = getpass.getpass('enter token')

enter token········


In [33]:
staging_server='https://staging-api.globalforestwatch.org'
prod_server = "https://api.resourcewatch.org"
app = ["aqueduct"]
env = 'production'
object_type = ['dataset']
search_term = ''

In [8]:
def testButton(widgetId):
    buttonUpdateL = widgets.Button(description="Update Layer")
    statusUpdateL = widgets.HTML(
    value="",
    placeholder='',
    description='',
    )
    buttonRevertL = widgets.Button(description="Revert Layer")
    statusRevertL = widgets.HTML(
    value="",
    placeholder='',
    description='',
    )
    layout_2x2 = widgets.TwoByTwoLayout(top_left=buttonUpdateL, 
                                        top_right=statusUpdateL,
                                        bottom_left=buttonRevertL, 
                                        bottom_right=statusRevertL,
                                        justify_items='left',
                                        width="100%",
                                        align_items="flex-start")
    CurrentWidget = GetWidget(widgetId)
    #New
    
    def on_update_layer(b):
        #print(status.children)
        statusUpdateL.value = f"Hello <b>Status: {CurrentWidget['data']['attributes']['dataset']} </b>"
        b.button_style = 'success'
        b.disabled=True
        buttonRevertL.disabled=False
        buttonRevertL.button_style = ''
        statusRevertL.value = ''
        return b
    def on_revert_layer(b):
        #print(status.children)
        UpdateWidget(CurrentWidget['data']['attributes']['dataset'], widgetId, {'widgetConfig':CurrentWidget['data']['attributes']['widgetConfig']})
        statusRevertL.value = f"Widget reverted"
        b.button_style = 'success'
        b.disabled=True
        buttonUpdateL.disabled=False
        buttonUpdateL.button_style = ''
        statusUpdateL.value = ''
        
        return b
        
    buttonUpdateL.on_click(on_update_layer)
    buttonRevertL.on_click(on_revert_layer)
    return layout_2x2

class Error(Exception):
    def __init__(self, m):
        self.message = m
    def __str__(self):
        return self.message

def GetWidget(widgetId: str):
    try:
        widgetUrl=f"{envUrl}/v1/widget/{widgetId}"
        s=requests.get(widgetUrl)
        if s.status_code == 200:
            return s.json()
        else:
                raise Error(f'status code {r.status_code}, {r.text}')
    except Error as e:
        raise e

        
def UpdateWidget(datasetId: str, widgetId: str, body: dict):
    try:
        url = f'{envUrl}/v1/dataset/{datasetId}/widget/{widgetId}'
        headers = {'Authorization': f'{OAUTH}', 'Content-Type': 'application/json'}
        r = requests.patch(url, data=json.dumps(body), headers=headers)
        if r.status_code == 200:
            return r.json()
        else:
            raise Error(f'status code {r.status_code}, {r.text}')
    except Error as e:
        raise e
        
def displayWidgetMetadata(name, widgetID, sqlOld, confOld, sqlNew, confNew, newVegaSpec={}, visualize=True):
    header = widgets.Output()
    with header:
        display(Markdown(f'#### {name}'))
        display(Markdown(f'**Widget ID**: [{widgetID}]({envUrl}/v1/widget/{widgetID})  '))
    
    accOld = widgets.Output()
    with accOld:
        display(Markdown(f""">```PLSQL {sqlOld} ``` """))
        display(Markdown(f""">```JSON
    {json.dumps(confOld, indent=2)}``` """))
        getData(sqlOld, confOld, testscenario)
        display(Markdown('---'))
    
    accNew = widgets.Output()
    with accNew:
        display(Markdown(f""">```PLSQL {sqlNew} ``` """))
        display(Markdown(f""">```JSON 
    {json.dumps(confNew, indent=2)} ``` """))
        display(Markdown('---'))
        display(Markdown('**Example result**'))
        getData(sqlNew, confNew, testscenario)
        
        
    accordionData = widgets.Accordion(children=[accOld, accNew])
    accordionData.set_title(0, 'Old data')
    accordionData.set_title(1, 'New data')
    accordionData.selected_index = 1
    tab_nest = widgets.Tab()
    tab_childs = [accordionData]
    visual = []
    if visualize:
        vegaView = widgets.Output()
        with vegaView:
            display(visualizeWidget(widgetID, confNew, testscenario))
        visual.append(vegaView)
    tab_childs.append(widgets.VBox(visual))
        
    tab_nest.children = tab_childs
    tab_nest.set_title(0, 'Query Information')
    tab_nest.set_title(1, 'Current api Visualization')

    app = widgets.AppLayout(header=header,
                    center=tab_nest,
                    pane_widths=[1, 5, 1],
                    pane_heights=[1, 5, 1],
                    width="100%",
                    height="100%",
                    align_items='top',
          grid_gap="1px")
    ####
    display(app)

testButton('9391e42c-f32b-40d9-a282-3083e617464f')

TwoByTwoLayout(children=(Button(description='Update Layer', layout=Layout(grid_area='top-left'), style=ButtonS…

NameError: name 'OAUTH' is not defined

In [34]:
def createMetadata(widgetId, body):
    try:
        headers = {
            'Authorization': f'{token}',
            'Content-Type': 'application/json'
        }
        datasetId = GetWidget(widgetId)['data']['attributes']['dataset']
        getData = requests.get(f'{staging_server}/v1/dataset/{datasetId}/metadata')
        if len(getData.json()['data']) > 0:
            r = requests.patch(f'{staging_server}/v1/dataset/{datasetId}/metadata', data=json.dumps(body), headers=headers)
            if r.status_code == 200:
                return r.json()
            else:
                
                raise Error(f'status code {r.status_code}, {r.url}, {r.text}')
        else:
            r = requests.post(f'{staging_server}/v1/dataset/{datasetId}/metadata', data=json.dumps(body), headers=headers)
            if r.status_code == 200:
                return r.json()
            else:
                raise Error(f'status code {r.status_code}, {r.text}')
    except Error as e:
        raise e

def GetWidget(widgetId: str):
    try:
        widgetUrl=f"{staging_server}/v1/widget/{widgetId}"
        s=requests.get(widgetUrl)
        if s.status_code == 200:
            return s.json()
        else:
            raise Error(f'status code {s.status_code}, {s.text}')
    except Error as e:
        raise e

In [37]:
widgetId = '606c813b-d716-4b0b-8717-9389425c9262'
body = {
    "application": "aqueduct",
    "language": "en",
    "name": "Percentage of global [crop] area that is irrigated vs. rainfed",
    "description": "This figure displays the share of harvested land that is irrigated versus rainfed for the selected crop(s) and timeframe.",
    "info": {
        "sources": [
            {
                "source-url": "https://www.ifpri.org/program/impact-model",
                "id": 0,
                "source-name": "IFPRI IMPACT Model 2015"
            }
        ]
    }
}
createMetadata(widgetId, body)

{'data': [{'id': '5a6073b001fa4b00134da6c3',
   'type': 'metadata',
   'attributes': {'dataset': 'fc85eac5-d27c-49d2-a248-dee26ec858bc',
    'application': 'aqueduct',
    'resource': {'id': 'fc85eac5-d27c-49d2-a248-dee26ec858bc',
     'type': 'dataset'},
    'language': 'en',
    'name': 'Percentage of global [crop] area that is irrigated vs. rainfed',
    'description': 'This figure displays the share of harvested land that is irrigated versus rainfed for the selected crop(s) and timeframe.',
    'source': '',
    'info': {'sources': [{'source-url': 'https://www.ifpri.org/program/impact-model',
       'id': 0,
       'source-name': 'IFPRI IMPACT Model 2015'}]},
    'createdAt': '2018-01-18T10:15:12.957Z',
    'updatedAt': '2019-11-20T15:25:52.431Z',
    'status': 'published'}}]}

In [35]:
widgetId = 'af969fd8-fd47-430d-ad13-9113e42e9fed'
body = {
   "application": "aqueduct",
   "language": "en",
    "name": "Kilocalories per capita in [Country] in [year]",
    "description": "This figure displays the selected country's average availability of kilocalories per person per day, benchmarked against a globally applied minimum requirement to prevent malnourishment (FAO 1996).",
    "info": {
        "sources": [
{
"source-url": "https://www.ifpri.org/program/impact-model",
"source-name": "IFPRI IMPACT Model 2015"
}
],
"units": "Kcal/person/day",
"resolution": "national"
    }
}
createMetadata( widgetId, body)

{'data': [{'id': '5a250cf46b2ad20012d9a289',
   'type': 'metadata',
   'attributes': {'dataset': '202c67a5-5370-4578-b8d6-16533fc44980',
    'application': 'aqueduct',
    'resource': {'id': '202c67a5-5370-4578-b8d6-16533fc44980',
     'type': 'dataset'},
    'language': 'en',
    'name': 'Kilocalories per capita in [Country] in [year]',
    'description': "This figure displays the selected country's average availability of kilocalories per person per day, benchmarked against a globally applied minimum requirement to prevent malnourishment (FAO 1996).",
    'info': {'sources': [{'source-url': 'https://www.ifpri.org/program/impact-model',
       'source-name': 'IFPRI IMPACT Model 2015'}],
     'units': 'Kcal/person/day',
     'resolution': 'national'},
    'columns': {},
    'createdAt': '2017-12-04T08:53:08.013Z',
    'updatedAt': '2019-11-20T11:26:28.821Z',
    'status': 'published'}}]}

In [46]:
widgetId = 'b759392b-96be-440f-80f5-c14929aac11b'
body = {
   "application": "aqueduct",
   "language": "en",
 "info": {
"sources": [
{
"source-url": "https://www.wri.org/publication/aqueduct-30",
"source-name": "WRI Aqueduct 2019"
},
{
"source-url": "http://mapspam.info/",
"source-name": "MapSPAM 2010"
}
]
}
}
createMetadata( widgetId, body)

{'data': [{'id': '5dd51fbb68b94e00103b7bad',
   'type': 'metadata',
   'attributes': {'dataset': '19fcbe42-69d7-48fb-a64b-ae1469dd14d3',
    'application': 'aqueduct',
    'resource': {'id': '19fcbe42-69d7-48fb-a64b-ae1469dd14d3',
     'type': 'dataset'},
    'language': 'en',
    'description': 'This figure displays the percentage of crop production by level of groundwater table decline for the timeframe selected.',
    'info': {'sources': [{'source-url': 'https://www.wri.org/publication/aqueduct-30',
       'source-name': 'WRI Aqueduct 2019'},
      {'source-url': 'http://mapspam.info/', 'source-name': 'MapSPAM 2010'}]},
    'createdAt': '2019-11-20T11:12:59.775Z',
    'updatedAt': '2019-11-20T15:41:19.322Z',
    'status': 'published'}}]}